In [3]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure

# ========= 1. Environment Wrapper =========
class EpisodicRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.cumulative_reward = 0.0

    def step(self, action):
        result = self.env.step(action)
        if len(result) == 5:
            obs, reward, terminated, truncated, info = result
            done = terminated or truncated
        else:
            obs, reward, done, info = result

        self.cumulative_reward += reward
        
        if done:
            reward = self.cumulative_reward
            self.cumulative_reward = 0.0
        else:
            reward = 0.0
        
        return obs, reward, done, info

    def reset(self, **kwargs):
        self.cumulative_reward = 0.0
        result = self.env.reset(**kwargs)
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        return obs



class TrajectoryReplay:
    def __init__(self):
        self.trajectories = []

    def add_trajectory(self, traj):
        self.trajectories.append(traj)

    def sample(self, batch_size):
        indices = np.random.randint(0, len(self.trajectories), size=batch_size)
        return [self.trajectories[i] for i in indices]

    def __len__(self):
        return len(self.trajectories)


class RewardModel(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )
        self.mu_layer = nn.Linear(hidden_size, 1)
        self.log_sigma_layer = nn.Linear(hidden_size, 1)

    def forward(self, s, a):
        x = torch.cat([s, a], dim=-1)
        x = self.net(x)
        mu = self.mu_layer(x)
        log_sigma = self.log_sigma_layer(x)
        sigma = torch.clamp(torch.exp(log_sigma), min=1e-3, max=4.0)
        return mu, sigma


# ========= 4. Train a reward model with batch trajectories ==========
def train_reward_model_gaussian_loo(reward_model, optimizer, trajectories, device='cpu'):
    reward_model.train()
    total_loss = 0.0
    total_episodes = 0

    for traj in trajectories:
        R_ep = sum([t[2] for t in traj])
        Tj = len(traj)
        if Tj == 0:
            continue

        s_list, a_list = [], []
        for (s, a, r, s_next, done) in traj:
            s_list.append(s)
            a_list.append(a)
        states = torch.tensor(s_list, dtype=torch.float, device=device)
        actions = torch.tensor(a_list, dtype=torch.float, device=device)
        
        mu, sigma = reward_model(states, actions)
        
        nll_list = []
        for i in range(Tj):
            mask = torch.ones(Tj, dtype=torch.bool, device=device)
            mask[i] = False
            epsilons = torch.randn((mask.sum(), 1), device=device)
            r_samples = mu[mask] + epsilons * sigma[mask]
            leave_one_sum = r_samples.sum()
            target_i = R_ep - leave_one_sum
            nll_i = torch.log(sigma[i]) + ((target_i - mu[i])**2) / (2 * sigma[i]**2)
            nll_list.append(nll_i)
        
        loss_traj = torch.stack(nll_list).mean()
        total_loss += loss_traj
        total_episodes += 1

    if total_episodes > 0:
        loss_mean = total_loss / total_episodes
    else:
        loss_mean = torch.tensor(0.0, device=device)

    optimizer.zero_grad()
    loss_mean.backward()
    optimizer.step()
    return loss_mean.item()








# ========= 5. How to collect a trajectory ==========
def collect_episodes(env, model, n_episodes, device='cpu'):
    trajectories = []
    for _ in range(n_episodes):
        result = env.reset()
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        done = False
        traj = []
        while not done:
            if obs is None or (hasattr(obs, '__len__') and len(obs) == 0):
                raise ValueError("Observation is empty, check env.reset() output")
            obs_tensor = torch.tensor(obs, dtype=torch.float, device=device).unsqueeze(0)
            with torch.no_grad():
                action, _states = model.predict(obs_tensor.cpu().numpy(), deterministic=False)
            result = env.step(action[0])
            
            if len(result) == 5:
                next_obs, reward, terminated, truncated, info = result
                done = terminated or truncated
            
            else:
                next_obs, reward, done, info = result
            traj.append((obs, action[0], reward, next_obs, done))
            obs = next_obs
        trajectories.append(traj)
    return trajectories

def add_shaped_transitions_to_replay(model, reward_model, trajectories, device='cpu'):
    for traj in trajectories:
        states = []
        actions = []
        next_states = []
        dones = []
        for (s, a, r, s_next, d) in traj:
            states.append(s)
            actions.append(a)
            next_states.append(s_next)
            dones.append(d)
        
        s_tensor = torch.tensor(states, dtype=torch.float, device=device)
        a_tensor = torch.tensor(actions, dtype=torch.float, device=device)
        
        with torch.no_grad():
            mu, _ = reward_model(s_tensor, a_tensor)
        shaped_rewards = mu.cpu().numpy().squeeze(-1)
        
        for i in range(len(traj)):
            s, a, _, s_next, d = traj[i]
            r_shaped = shaped_rewards[i]
            model.replay_buffer.add(
                s, s_next, a, r_shaped, d, infos=[{}],
            )

def main():
    class Args:
        env = "BipedalWalker-v3"
        episodes = 8000
        steps_per_update = 200
        device = "cuda"
    
    args = Args()
    device = torch.device(args.device if torch.cuda.is_available() else "cpu")

    base_env = gym.make(args.env)
    env = EpisodicRewardWrapper(base_env)
    vec_env = DummyVecEnv([lambda: env])

    policy_kwargs = dict(
        net_arch=dict(pi=[64, 64], qf=[64, 64])
    )
    model = SAC(
        policy="MlpPolicy",
        env=vec_env,
        policy_kwargs=policy_kwargs,
        verbose=1,
        seed=42,
        buffer_size=100000,
        learning_starts=0,
        train_freq=1,
        gradient_steps=0,
        batch_size=512,
        gamma=0.99,
        device=device
    )

    from stable_baselines3.common.logger import configure
    model._logger = configure(folder=None, format_strings=["stdout"])

    state_dim = base_env.observation_space.shape[0]
    action_dim = base_env.action_space.shape[0]
    reward_model = RewardModel(state_dim, action_dim).to(device)
    reward_optimizer = optim.Adam(reward_model.parameters(), lr=3e-4)

    traj_replay = TrajectoryReplay()

    ep_record = []
    for ep in tqdm(range(args.episodes)):
        new_trajectories = collect_episodes(env, model, n_episodes=1, device=device)
        for traj in new_trajectories:
            traj_replay.add_trajectory(traj)

        if len(traj_replay) >= 5:
            batch_size_trajectories = 4
            sampled_trajs = traj_replay.sample(batch_size_trajectories)
            loss_val = train_reward_model_gaussian_loo(
                reward_model, reward_optimizer, sampled_trajs, device=device
            )
        
        add_shaped_transitions_to_replay(model, reward_model, new_trajectories, device=device)
        
        if len(traj_replay) >= 5:
            model.train(args.steps_per_update)
        ep_return = sum([t[2] for t in new_trajectories[-1]])
        ep_record.append(ep_return)
        if ep % 10 == 0:
            print(f"Episode {ep+1}/{args.episodes}, real env return = {ep_return:.2f}")

    print("Training finished.")
    return ep_record

In [4]:
train1 = main()

Using cuda device


  0%|                                          | 2/8000 [00:00<15:39,  8.51it/s]

Episode 1/8000, real env return = -103.75


  0%|                                      | 11/8000 [00:39<13:12:47,  5.95s/it]

Episode 11/8000, real env return = -99.19


...

Episode 7981/8000, real env return = 283.65


100%|████████████████████████████████████▉| 7991/8000 [7:34:00<00:21,  2.40s/it]

Episode 7991/8000, real env return = 284.69


100%|█████████████████████████████████████| 8000/8000 [7:34:18<00:00,  3.41s/it]


Training finished.


In [6]:
train3 = main()

Using cuda device


  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Episode 1/8000, real env return = -103.57


  0%|                                       | 11/8000 [00:14<4:01:20,  1.81s/it]

Episode 11/8000, real env return = -82.25


...

Episode 7981/8000, real env return = 307.08


100%|████████████████████████████████████▉| 7991/8000 [4:52:07<00:20,  2.28s/it]

Episode 7991/8000, real env return = 307.41


100%|█████████████████████████████████████| 8000/8000 [4:52:25<00:00,  2.19s/it]


Training finished.


In [10]:
train2 = main()

Using cuda device


  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Episode 1/8000, real env return = -104.06


  0%|                                       | 11/8000 [00:15<3:58:17,  1.79s/it]

Episode 11/8000, real env return = -84.88


...

Episode 7981/8000, real env return = -38.67


100%|████████████████████████████████████▉| 7991/8000 [6:13:17<00:25,  2.88s/it]

Episode 7991/8000, real env return = -17.18


100%|█████████████████████████████████████| 8000/8000 [6:13:39<00:00,  2.80s/it]


Training finished.


In [11]:
train4 = main()

Using cuda device


  0%|                                          | 2/8000 [00:00<08:38, 15.42it/s]

Episode 1/8000, real env return = -103.57


  0%|                                       | 11/8000 [00:08<2:37:37,  1.18s/it]

Episode 11/8000, real env return = -112.95


...

Episode 7981/8000, real env return = -20.99


100%|████████████████████████████████████▉| 7991/8000 [6:59:11<00:29,  3.27s/it]

Episode 7991/8000, real env return = -15.72


100%|█████████████████████████████████████| 8000/8000 [6:59:41<00:00,  3.15s/it]


Training finished.


In [12]:
train5 = main()

Using cuda device


  0%|                                                  | 0/8000 [00:00<?, ?it/s]

Episode 1/8000, real env return = -103.58


  0%|                                       | 11/8000 [00:13<3:36:45,  1.63s/it]

Episode 11/8000, real env return = -93.20


...

Episode 7981/8000, real env return = 292.47


100%|████████████████████████████████████▉| 7991/8000 [7:02:46<00:24,  2.76s/it]

Episode 7991/8000, real env return = 294.76


100%|█████████████████████████████████████| 8000/8000 [7:03:11<00:00,  3.17s/it]


Training finished.
